In [36]:
# for tree calculator
import pandas as pd
zipdf = pd.read_csv('./tn_zips_rvals.csv') # zip codes and corresponding counties and lat/long coords
# tree csv
treedf = pd.read_csv('./treeinfo.csv')
# native plant csv
plantsdf = pd.read_csv('./nativeplants.csv')

zipcode = 37214


# utils for printing out the results
def print_species_list(species_list):
    if len(species_list) == 0:
        return "There are no species planted."
    elif len(species_list) == 1:
        return "The most common species planted was " + species_list[0] + "."
    elif len(species_list) == 2:
        return "The most common species planted were " + species_list[0] + " and " + species_list[1] + "."
    else:
        species_str = ", ".join(species_list[:-1])
        return "The most common species planted were " + species_str + ", and " + species_list[-1] + "."




# get county name (without county at the end) from zipcode
county = zipdf.loc[zipdf['zip']==zipcode,'county'].values[0].split()[0] # "Davidson County" -> "Davidson"
county

'Davidson'

In [37]:
treedf.head()

,County,SpeciesName,CommonName,CarbonStoragelb,GrossCarbonSequestrationLBYR,AvoidedRunoffGYR,PollutionRemovalOZYR,OxygenProductionLBYR,TotalAnnualBenefitsDYR
0,Anderson,Ilex cookii,Te,3.0,0.9,0.6,0.4,2.4,0.11
1,Anderson,Ilex cookii,Te,3.0,0.9,0.6,0.4,2.4,0.11
2,Anderson,Ilex cookii,Te,3.0,0.9,0.6,0.4,2.4,0.11
3,Anderson,Ilex cookii,Te,3.0,0.9,0.6,0.4,2.4,0.11
4,Anderson,Ilex cookii,Te,3.0,0.9,0.6,0.4,2.4,0.11


In [38]:
# county="Doodoo"
if not county in treedf['County'].unique():
    print("County not in database")
else:
    countytrees = treedf.loc[treedf['County']==county,]

    # Sentence about number of trees
    ntrees = f"There are a total of {len(countytrees)} reported trees planted in TNSY certified yards in {county} county. " 
    # Sentence about most common species
    topthreespecies = print_species_list(list(countytrees['SpeciesName'].value_counts().nlargest(3).index))

print(ntrees + '\n' + topthreespecies)

There are a total of 231 reported trees planted in TNSY certified yards in Davidson county. 
The most common species planted were Amelanchier, Cercis, and Asimina triloba.


In [39]:
estbenefits = "Estimated Benefits: \n" +\
    f"{countytrees['GrossCarbonSequestrationLBYR'].sum():.1f} lbs of carbon sequestered per year \n" +\
    f"{countytrees['CarbonStoragelb'].sum():.1f} lbs of carbon stored* \n" +\
    f"{countytrees['AvoidedRunoffGYR'].sum():.1f} gallons of runoff avoided per year \n" +\
    f"{countytrees['PollutionRemovalOZYR'].sum():.1f} ounces of pollution removed per year \n" +\
    f"{countytrees['OxygenProductionLBYR'].sum():.1f} pounds of oxygen produced per year \n" +\
    f"Total annual benefits of carbon sequestration, avoided runoff, and pollution removal amounts to ${countytrees['TotalAnnualBenefitsDYR'].sum():.2f} per year! "
print(estbenefits)

Estimated Benefits: 
238.2 lbs of carbon sequestered per year 
786.9 lbs of carbon stored* 
1818.5 gallons of runoff avoided per year 
110.1 ounces of pollution removed per year 
634.1 pounds of oxygen produced per year 
Total annual benefits of carbon sequestration, avoided runoff, and pollution removal amounts to $47.67 per year! 


In [40]:
plantsdf.head()

,Zip_Code,Improvement,Hydrologic_Soil_Group,Sq_Ft,Residential,Meadow
0,37027,Yes,clay_loam,800,0.43,0.35
1,37027,Yes,clay_loam,1000,0.43,0.35
2,37027,Yes,clay_loam,500,0.43,0.35
3,37027,Yes,clay_loam,1000,0.43,0.35
4,37027,Yes,clay_loam,2000,0.43,0.35


In [45]:
plantsdf['Zip_Code'].value_counts()

Zip_Code
37027    5
37774    5
37211    5
37934    4
37216    4
        ..
37415    1
37416    1
37032    1
37801    1
38572    1
Name: count, Length: 79, dtype: int64

In [46]:
zipplants = plantsdf.loc[plantsdf['Zip_Code']==37027]
len(zipplants)

5

In [81]:
import rasterio
import rasterio.warp
import pandas as pd

tnzips = pd.read_csv('./tn_zipcodes.csv')

In [83]:
def getRfac(zipcode):
    lat = tnzips.loc[tnzips['zip']==zipcode,'latitude'].values[0]
    long = tnzips.loc[tnzips['zip']==zipcode,'longitude'].values[0]
        # Open your raster file
    # dataset = rasterio.open('./tn_rfactors.tif')
    with rasterio.open('./Rfactors.tif') as dataset:
        # convert decimal degrees (global coordinates) into dataset's CRS
        # Define the target CRS. the CRS of the raster dataset
        target_crs = dataset.crs

        # Transform the global coordinates to the dataset's CRS
        x, y = rasterio.warp.transform(src_crs="EPSG:4326", dst_crs=target_crs, xs=[long], ys=[lat])

        # Sample the pixel value at the transformed coordinates
        for val in dataset.sample([(x[0], y[0])], indexes=1):
            rval = val[0]
    return rval

In [84]:
tnzips['rfactor'] = tnzips.apply(lambda row: getRfac(row['zip']), axis=1)

In [85]:
tnzips.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population,rfactor
0,37010,STANDARD,0,Adams,NaN,NaN,TN,Montgomery County,America/Chicago,"270, 364, 615, 629, 931",NaN,US,36.58,-87.06,5680,219.189178
1,37011,PO BOX,0,Antioch,NaN,NaN,TN,Davidson County,America/Chicago,615,NaN,US,36.04,-86.64,1107,224.257599
2,37012,STANDARD,0,Alexandria,NaN,NaN,TN,DeKalb County,America/Chicago,"615, 629",NaN,US,36.07,-86.03,2110,214.633240
3,37013,STANDARD,0,Antioch,Cane Ridge,NaN,TN,Davidson County,America/Chicago,"615, 629",NaN,US,36.04,-86.64,85580,224.257599
4,37014,STANDARD,0,Arrington,NaN,Triune,TN,Williamson County,America/Chicago,"615, 629",NaN,US,35.87,-86.63,2800,226.952332


In [82]:

    # Open your raster file
    # dataset = rasterio.open('./tn_rfactors.tif')
    with rasterio.open(dir_path + '/tn_rfactors.tif') as dataset:
        # convert decimal degrees (global coordinates) into dataset's CRS
        # Define the target CRS. the CRS of the raster dataset
        target_crs = dataset.crs

        # Transform the global coordinates to the dataset's CRS
        x, y = rasterio.warp.transform(src_crs="EPSG:4326", dst_crs=target_crs, xs=[long], ys=[lat])

        # Sample the pixel value at the transformed coordinates
        for val in dataset.sample([(x[0], y[0])], indexes=1):
            rval = val[0]

array([37010, 37011, 37012, 37013, 37014, 37015, 37016, 37018, 37019,
       37020, 37022, 37023, 37024, 37025, 37026, 37027, 37028, 37029,
       37030, 37031, 37032, 37033, 37034, 37035, 37036, 37037, 37040,
       37041, 37042, 37043, 37044, 37046, 37047, 37048, 37049, 37050,
       37051, 37052, 37055, 37056, 37057, 37058, 37059, 37060, 37061,
       37062, 37063, 37064, 37065, 37066, 37067, 37068, 37069, 37070,
       37071, 37072, 37073, 37074, 37075, 37076, 37077, 37078, 37079,
       37080, 37082, 37083, 37085, 37086, 37087, 37088, 37089, 37090,
       37091, 37095, 37096, 37097, 37098, 37101, 37110, 37111, 37115,
       37116, 37118, 37119, 37121, 37122, 37127, 37128, 37129, 37130,
       37131, 37132, 37133, 37134, 37135, 37136, 37137, 37138, 37140,
       37141, 37142, 37143, 37144, 37145, 37146, 37148, 37149, 37150,
       37151, 37152, 37153, 37160, 37161, 37162, 37165, 37166, 37167,
       37171, 37172, 37174, 37175, 37178, 37179, 37180, 37181, 37183,
       37184, 37185,

In [1]:
import rasterio
import rasterio.features
import rasterio.warp

location = {'latitude':36.16913,'longitude':-86.78479}
location

# Open your raster file
dataset = rasterio.open('./TN_Rfactors.tif')
with rasterio.open('./TN_Rfactors.tif') as dataset:
    # convert decimal degrees (global coordinates) into dataset's CRS
    # Define the target CRS. the CRS of the raster dataset
    target_crs = dataset.crs

    # Transform the global coordinates to the dataset's CRS
    x, y = rasterio.warp.transform(src_crs="EPSG:4326", dst_crs=target_crs, xs=[location['longitude']], ys=[location['latitude']])

    # Sample the pixel value at the transformed coordinates
    for val in dataset.sample([(x[0], y[0])], indexes=1):
        rval = val[0]

In [50]:
import rasterio
import rasterio.features
import rasterio.warp

location = {'latitude':36.16913,'longitude':-86.78479}
location

# Open your raster file
dataset = rasterio.open('./TN_Rfactors.tif')
source_crs = dataset.crs 
target_crs = "EPSG:4326"

transform, width, height = rasterio.warp.calculate_default_transform(source_crs, target_crs, dataset.width, dataset.height, *dataset.bounds)
kwargs = dataset.meta.copy()
kwargs.update({
    'crs': target_crs,
    'transform': transform,
    'width': width,
    'height': height})

output = rasterio.open('./TN_Rfactors_EPSG4326.tif','w',**kwargs)
for i in range(1, dataset.count + 1):
    rasterio.warp.reproject(
        source=rasterio.band(dataset, i),
        destination=rasterio.band(output, i),
        src_transform=dataset.transform,
        src_crs=dataset.crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=rasterio.warp.Resampling.nearest)
    
print(output)


<open DatasetWriter name='./TN_Rfactors_EPSG4326.tif' mode='w'>


In [75]:
from osgeo import gdal
import os

location = {'latitude':36.16913,'longitude':-86.78479}
location
lat, long = location['latitude'], location['longitude']
# Open your raster file
raster_file = gdal.Open('./TN_Rfactors_EPSG4326.tif')
prj=raster_file.GetProjection()
print(prj)

# # Get the raster band (1 in this case)
# band = raster_file.GetRasterBand(1)

# # Define your coordinate (longitude, latitude)

# x, y = location['latitude'], location['longitude']

# # # Convert the coordinate to the raster's georeferenced space
# # transform = raster_file.GetGeoTransform()
# # xOrigin = transform[0]
# # yOrigin = transform[3]
# # pixelWidth = transform[1]
# # pixelHeight = -transform[5]

# # # Compute the pixel offset
# # xOffset = int((x - xOrigin) / pixelWidth)
# # yOffset = int((y - yOrigin) / pixelHeight)

# # # Read the value at the computed offset
# # value = band.ReadAsArray(x, y, 1, 1)

# result = os.popen('gdallocationinfo -valonly -wgs84 %s %s %s' % (band, lat, long)).read()


# # Close the raster file
# raster_file = None

# # Print the value
# # print("The value at the given coordinate is:", value[0][0])
# print(result)




GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


In [79]:
from osgeo import gdal_array
rasterArray = gdal_array.LoadFile('./Rfactors.tif')
rasterArray
# rasterArray[lat,long]
# print(lat)

array([[245.81409, 245.90125, 245.98593, ..., 138.67458, 138.69199,
        138.71002],
       [246.07236, 246.15802, 246.24124, ..., 138.96762, 138.98761,
        139.00821],
       [246.32693, 246.41093, 246.49254, ..., 139.26382, 139.28629,
        139.30933],
       ...,
       [299.1938 , 299.16254, 299.13452, ..., 277.1869 , 277.22278,
        277.25848],
       [299.37094, 299.3375 , 299.30954, ..., 277.31598, 277.35117,
        277.38608],
       [299.5432 , 299.50842, 299.48224, ..., 277.4441 , 277.4786 ,
        277.51285]], dtype=float32)

In [ ]:
dataset = rasterio.open('./TN_Rfactors_.tif')


In [45]:
dataset = rasterio.open('./TN_Rfactors.tif')
print("Number of bands in the dataset: " + str(dataset.count))
print("Width of the dataset: " + str(dataset.width))
print("Height of the dataset: " + str(dataset.height))

Number of bands in the dataset: 1
Width of the dataset: 926
Height of the dataset: 325


In [46]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'float32'}

In [49]:
dataset.crs


CRS.from_epsg(5070)

In [50]:
location = {'latitude':36.16913,'longitude':-86.78479}
location

{'latitude': 36.16913, 'longitude': -86.78479}

In [58]:
# convert decimal degrees (global coordinates) into dataset's CRS

# Define the target CRS. For example, the CRS of the raster dataset
target_crs = dataset.crs

# Transform the global coordinates to the dataset's CRS
x, y = rasterio.warp.transform(src_crs="EPSG:4326", dst_crs=target_crs, xs=[location['longitude']], ys=[location['latitude']])

# Sample the pixel value at the transformed coordinates
for val in dataset.sample([(x[0], y[0])], indexes=1):
    print(val[0])

223.68724


In [17]:
# # Reference: https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/fundamentals-raster-data/intro-to-the-geotiff-file-format/

# # Import necessary packages
# import os
# import rioxarray as rxr
# # import earthpy as et
# import pandas as pd 

# datatif = rxr.open_rasterio("R-Factor-CONUS/R-Factor_CONUS.tif",
#                                  masked=True)
# datatif
# datatif.rio.bounds()
# # View generate metadata associated with the raster file
# print("The crs of your data is:", datatif.rio.crs)
# print("The nodatavalue of your data is:", datatif.rio.nodata)
# print("The shape of your data is:", datatif.shape)
# print("The spatial resolution for your data is:", datatif.rio.resolution())
# print("The metadata for your data is:", datatif.attrs)

In [18]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

In [19]:
# Function to geocode an address
def get_coordinates(address):
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

In [34]:
# address = "952 wemberton dr nashville tn"
# get_coordinates(address)

In [35]:
# lat and long of davidson county
coordinates = {'latitude':36.16913,'longitude':-86.78479}
coordinates

{'latitude': 36.16913, 'longitude': -86.78479}

In [36]:
# Open your raster file
# raster_file = gdal.Open('./Rfactors.tif')
raster_file = gdal.Open('./R-Factor-CONUS/R-Factor_CONUS.tif') 


# Get the raster band (1 in this case)
band = raster_file.GetRasterBand(1)

# Define your coordinate (longitude, latitude)

x, y = coordinates['latitude'], coordinates['longitude']

# Convert the coordinate to the raster's georeferenced space
transform = raster_file.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = -transform[5]

# Compute the pixel offset
xOffset = int((x - xOrigin) / pixelWidth)
yOffset = int((y - yOrigin) / pixelHeight)

# Read the value at the computed offset
value = band.ReadAsArray(xOffset, yOffset, 1, 1)

# Close the raster file
raster_file = None

# Print the value
print("The value at the given coordinate is:", value[0][0])

ERROR 5: R-Factor_CONUS.tif, band 1: Access window out of range in RasterIO().  Requested
(2998,-4010) of size 1x1 on raster of 5874x3677.


TypeError: 'NoneType' object is not subscriptable

In [37]:
yOffset

-4011

In [3]:
table = {
    0.2: [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,\
        0.05, 0.05, 0.05, 0.06, 0.06],
    0.5: [0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.08, 0.08, 0.08, 0.09, 0.09, 0.09,\
        0.09, 0.09, 0.09, 0.1, 0.1],
    1: [0.11, 0.11, 0.11, 0.11, 0.12, 0.13, 0.14, 0.14, 0.16, 0.17, 0.18, 0.18, \
      0.19, 0.2, 0.2, 0.2, 0.2],
    2: [0.17, 0.17, 0.17, 0.17, 0.19, 0.22, 0.25, 0.27, 0.29, 0.31, 0.33, 0.33, \
      0.35, 0.37, 0.41, 0.44, 0.47],
    3: [0.22, 0.22, 0.22, 0.22, 0.25, 0.32, 0.4, 0.47, 0.53, 0.58, 0.62, 0.62, 0.69,\
      0.74, 0.86, 0.99, 1.1],
    4: [0.26, 0.26, 0.26, 0.26, 0.31, 0.4, 0.49, 0.58, 0.65, 0.72, 0.77, 0.77, 0.8,\
      0.86, 0.99, 1.1, 1.19],
    5: [0.3, 0.3, 0.3, 0.3, 0.37, 0.49, 0.58, 0.65, 0.72, 0.85, 0.93, 0.93, 1.01,\
      1.13, 1.33, 1.49, 1.63],
    6: [0.34, 0.34, 0.34, 0.34, 0.43, 0.58, 0.74, 0.91, 1.04, 1.16, 1.25, 1.25,\
      1.31, 1.42, 1.69, 1.91, 2.11],
    8: [0.42, 0.42, 0.42, 0.42, 0.53, 0.74, 0.97, 1.19, 1.38, 1.62, 1.77, 1.77,\
      1.89, 2.03, 2.47, 2.83, 3.15],
    10: [0.46, 0.48, 0.5, 0.51, 0.67, 0.97, 1.3, 1.53, 1.79, 2.15, 2.44, 2.44, 2.61,\
       2.84, 3.5, 4.06, 4.56],
    12: [0.47, 0.53, 0.58, 0.61, 0.84, 1.23, 1.61, 1.99, 2.23, 2.61, 2.95, 2.95,\
       3.26, 3.81, 4.75, 5.56, 6.28],
    14: [0.48, 0.58, 0.65, 0.7, 1, 1.48, 2, 2.48, 2.84, 3.25, 3.69, 3.69, 4.09,\
       4.82, 6.07, 7.15, 8.11],
    16: [0.49, 0.63, 0.72, 0.79, 1.15, 1.73, 2.28, 2.82, 3.26, 3.9, 4.45, 4.45,\
       4.95, 5.86, 7.43, 8.79, 9.93],
    20: [0.52, 0.71, 0.85, 0.96, 1.45, 2.22, 2.85, 3.45, 3.97, 4.9, 5.4, 5.4, 5.97,\
       7.97, 10.23, 12.2, 13.9],
    25: [0.56, 0.81, 1, 1.16, 1.81, 2.82, 3.65, 4.39, 5.34, 6.83, 7.88, 7.88, 8.86,\
       10.65, 13.3, 15.54, 19.13],
    30: [0.59, 0.89, 1.13, 1.34, 2.15, 3.39, 4.42, 5.34, 6.24, 7.88, 9.76, 9.76, 11.01,\
       13.3, 17.37, 20.99, 24.31],
    40: [0.65, 1.05, 1.38, 1.68, 2.77, 4.45, 5.87, 7.14, 8.11, 11.47, 13.37, 13.37,\
       15.14, 18.43, 23.17, 29.6, 34.48],
    50: [0.71, 1.18, 1.59, 1.97, 3.32, 5.4, 7.2, 8.78, 10.23, 14.26, 16.67, 16.67,\
       18.94, 23.17, 30.78, 37.65, 44.02],
    60: [0.76, 1.3, 1.78, 2.23, 3.81, 6.24, 8.33, 10.23, 11.01, 16.63, 19.64, 19.64,\
       22.36, 27.45, 36.63, 44.96, 52.7]
}

# List of slope lengths corresponding to the columns in the table
slope_lengths = [3, 6, 9, 12, 15, 25, 50, 75, 100, 150, 200,\
                 250, 300, 400, 600, 800, 1000]


def find_nearest_value(input_value, available_values):
  return min(available_values, key=lambda x: abs(x - input_value))

find_nearest_value(43,table.keys())

40

In [25]:
import json
with open("ls_by_slopepercent.json", "w") as outfile: 
    json.dump(table, outfile)

In [5]:
find_nearest_value(90,table.keys())

60

In [40]:
import pandas as pd

df = pd.DataFrame(table)
# slopelens = pd.Series(slope_lengths, name="slope length")
# df = df.join(slopelens)
# # df = pd.concat([df,slopelens],axis=1)
slope_percentages = df.columns
print(slope_percentages)
df.insert(loc=0, column='slope_lengths', value=slope_lengths)

# tdf = df.T
# tdf.columns = [3, 6, 9, 12, 15, 25, 50, 75, 100, 150, 200,\
#                  250, 300, 400, 600, 800, 1000]
# tdf
df

Index([ 0.2,  0.5,  1.0,  2.0,  3.0,  4.0,  5.0,  6.0,  8.0, 10.0, 12.0, 14.0,
       16.0, 20.0, 25.0, 30.0, 40.0, 50.0, 60.0],
      dtype='float64')


,slope_lengths,0.2,0.5,1.0,2.0,3.0,4.0,5.0,6.0,8.0,10.0,12.0,14.0,16.0,20.0,25.0,30.0,40.0,50.0,60.0
0,3,0.05,0.07,0.11,0.17,0.22,0.26,0.30,0.34,0.42,0.46,0.47,0.48,0.49,0.52,0.56,0.59,0.65,0.71,0.76
1,6,0.05,0.07,0.11,0.17,0.22,0.26,0.30,0.34,0.42,0.48,0.53,0.58,0.63,0.71,0.81,0.89,1.05,1.18,1.30
2,9,0.05,0.07,0.11,0.17,0.22,0.26,0.30,0.34,0.42,0.50,0.58,0.65,0.72,0.85,1.00,1.13,1.38,1.59,1.78
3,12,0.05,0.07,0.11,0.17,0.22,0.26,0.30,0.34,0.42,0.51,0.61,0.70,0.79,0.96,1.16,1.34,1.68,1.97,2.23
4,15,0.05,0.07,0.12,0.19,0.25,0.31,0.37,0.43,0.53,0.67,0.84,1.00,1.15,1.45,1.81,2.15,2.77,3.32,3.81
5,25,0.05,0.07,0.13,0.22,0.32,0.40,0.49,0.58,0.74,0.97,1.23,1.48,1.73,2.22,2.82,3.39,4.45,5.40,6.24
6,50,0.05,0.08,0.14,0.25,0.40,0.49,0.58,0.74,0.97,1.30,1.61,2.00,2.28,2.85,3.65,4.42,5.87,7.20,8.33
7,75,0.05,0.08,0.14,0.27,0.47,0.58,0.65,0.91,1.19,1.53,1.99,2.48,2.82,3.45,4.39,5.34,7.14,8.78,10.23
8,100,0.05,0.08,0.16,0.29,0.53,0.65,0.72,1.04,1.38,1.79,2.23,2.84,3.26,3.97,5.34,6.24,8.11,10.23,11.01
9,150,0.05,0.09,0.17,0.31,0.58,0.72,0.85,1.16,1.62,2.15,2.61,3.25,3.90,4.90,6.83,7.88,11.47,14.26,16.63


In [41]:
# tdf.to_csv('ls_by_slopepercent.csv')


In [45]:
lsdf = pd.melt(df, id_vars=['slope_lengths'], value_vars=list(slope_percentages),var_name='slope_percents', value_name='ls')
lsdf

,slope_lengths,slope_percents,ls
0,3,0.2,0.05
1,6,0.2,0.05
2,9,0.2,0.05
3,12,0.2,0.05
4,15,0.2,0.05
...,...,...,...
318,300,60.0,22.36
319,400,60.0,27.45
320,600,60.0,36.63
321,800,60.0,44.96


In [46]:
lsdf.to_csv('ls_values.csv')
